# Importing lib

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

#  Loading & Preparing data

In [ ]:
train = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')


In [ ]:
train.info()

In [ ]:
train

In [ ]:
data = train.copy()

In [ ]:
# making segments data into one data frame
def make_data():
    global data
    n = 0
    for  i in data.segment_id :
        file = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train/{}.csv'.format(i))
        for x in file.columns:
            data.loc[n:n+1,x+'_mean'] = file[x].mean()
            data.loc[n,x+'_std'] = file[x].std()
            data.loc[n,x+'_min'] = file[x].min()
            data.loc[n,x+'_20'] = file[x].quantile(0.20)
            data.loc[n,x+'_40'] = file[x].quantile(0.40)
            data.loc[n,x+'_50'] = file[x].quantile(0.50)
            data.loc[n,x+'_60'] = file[x].quantile(0.60)
            data.loc[n,x+'_80'] = file[x].quantile(0.80)
            data.loc[n,x+'_max'] = file[x].max()
        n +=1

    data.fillna(0,inplace=True)
    return data
make_data()    

In [ ]:
data_train = data.copy()

In [ ]:
data_train.info()

In [ ]:
data_train.describe()

# Data Preparation

In [ ]:
y_train = data_train['time_to_eruption']
segment_id = data_train['segment_id']
data_train.drop(['time_to_eruption','segment_id'],axis = 1 , inplace=True)

In [ ]:
# finding and removing constant columns
cols = []
for i in data_train.columns :
    if data_train[i].min() == data_train[i].max():
        cols.append(i)
cols

In [ ]:
data_train.drop(['sensor_1_50', 'sensor_2_50', 'sensor_3_50','sensor_4_50', 'sensor_5_50', 'sensor_6_50',
                 'sensor_7_50', 'sensor_8_50', 'sensor_9_50', 'sensor_10_50'],axis = 1 , inplace=True)

In [ ]:
data_train.shape

In [ ]:
std_scaler = StandardScaler()
std_data = std_scaler.fit_transform(data_train)
data_train = pd.DataFrame(std_data, columns = data_train.columns)
data_train

# Training Models

### 1. LinearRegression

In [ ]:
reg_model = LinearRegression()
reg_model.fit(data_train,y_train)
print(reg_model.score(data_train,y_train))


### 2. RandomForestRegressor

In [ ]:
forest_model = RandomForestRegressor()
forest_model.fit(data_train,y_train)
print(forest_model.score(data_train,y_train))


In [ ]:
forest_score = cross_val_score(forest_model,data_train,y_train,scoring= 'neg_mean_squared_error', cv=20)
forest_rmse_score = np.sqrt(-forest_score)
print(forest_rmse_score.mean())



### 3. DecisionTreeRegressor

In [ ]:
tree_model = DecisionTreeRegressor()
tree_model.fit(data_train,y_train)
print(tree_model.score(data_train,y_train))


In [ ]:
tree_score = cross_val_score(tree_model,data_train,y_train,scoring= 'neg_mean_squared_error', cv=20)
tree_rmse_score = np.sqrt(-tree_score)
print(tree_rmse_score.mean())



### 4. SVR

In [ ]:
svr_model = SVR()
svr_model.fit(data_train,y_train)
print(svr_model.score(data_train,y_train))


### 5. GradientBoostingRegressor

In [ ]:
gbr_model = GradientBoostingRegressor()
gbr_model.fit(data_train,y_train)
print(gbr_model.score(data_train,y_train))

In [ ]:
gbr_score = cross_val_score(gbr_model,data_train,y_train,scoring= 'neg_mean_squared_error', cv=20)
gbr_rmse_score = np.sqrt(-gbr_score)
print(gbr_rmse_score.mean())


### 6. xgb

In [ ]:
xgb_model = xgb.XGBRegressor()
xgb_model.fit(data_train,y_train)
print(xgb_model.score(data_train,y_train))

In [ ]:
xgb_score = cross_val_score(xgb_model,data_train,y_train,scoring= 'neg_mean_squared_error', cv=20)
xgb_rmse_score = np.sqrt(-xgb_score)
print(xgb_rmse_score.mean())


### 7. lgb

In [ ]:
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(data_train,y_train)
print(lgb_model.score(data_train,y_train))

In [ ]:
lgb_score = cross_val_score(lgb_model,data_train,y_train,scoring= 'neg_mean_squared_error', cv=20)
lgb_rmse_score = np.sqrt(-lgb_score)
print(lgb_rmse_score.mean())


# FINE TUNE THE MODELS


### 1. RandomForestRegressor

In [ ]:
forest_tune_pipeline = Pipeline([
     ('selector',SelectKBest(f_regression)),
     ('model',RandomForestRegressor(random_state = 42))])

forest_grid_search = GridSearchCV( estimator = forest_tune_pipeline, param_grid = {'selector__k':[70] , 
  'model__n_estimators':np.arange(500,550,50),'model__max_depth':[15]}, n_jobs=-1,
                                  scoring="neg_mean_squared_error", cv=20, verbose=3)

In [ ]:
forest_grid_search.fit(data_train,y_train)
print('the best parameters : ',forest_grid_search.best_params_)
print('the best score = ', np.sqrt(-forest_grid_search.best_score_))

### 2. DecisionTreeRegressor

In [ ]:
tree_tune_pipeline = Pipeline([
     ('selector',SelectKBest(f_regression)),
     ('model',DecisionTreeRegressor(random_state = 42))])

tree_grid_search = GridSearchCV( estimator = tree_tune_pipeline, param_grid = {'selector__k':[70,80] , 
     'model__max_depth':[12,13,15]}, n_jobs=-1,scoring="neg_mean_squared_error", cv=20, verbose=3)

In [ ]:
tree_grid_search.fit(data_train,y_train)
print('the best parameters : ',tree_grid_search.best_params_)
print('the best score = ', np.sqrt(-tree_grid_search.best_score_))

### 3. xgb

In [ ]:
xgb_tune_pipeline = Pipeline([
     ('selector',SelectKBest(f_regression)), ('model',xgb.XGBRegressor(random_state=42))])

xgb_grid_search = GridSearchCV( estimator = xgb_tune_pipeline, param_grid = {'selector__k':[70] , 
  'model__learning_rate':[0.05],'model__n_estimators':[3000,5000],'model__max_depth':[10],'model__colsample_bytree':[0.3]},
                               n_jobs=-1, scoring="neg_mean_squared_error", cv=20, verbose=3)

In [ ]:
xgb_grid_search.fit(data_train,y_train)
print('the best parameters : ',xgb_grid_search.best_params_)
print('the best score = ', np.sqrt(-xgb_grid_search.best_score_))

### 4. lgb

In [ ]:
lgb_tune_pipeline = Pipeline([
     ('selector',SelectKBest(f_regression)),
     ('model',lgb.LGBMRegressor(random_state=42,objective='regression',
                              bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11))])

lgb_grid_search = GridSearchCV( estimator = lgb_tune_pipeline, param_grid = {'selector__k':[70] , 
  'model__learning_rate':[0.01],'model__num_iterations':[10000],'model__n_estimators':[500],
                            'model__max_bin':[100],'model__num_leaves':[40,50]},
                               n_jobs=-1, scoring="neg_mean_squared_error", cv=20, verbose=3)

In [ ]:
lgb_grid_search.fit(data_train,y_train)
print('the best parameters : ',lgb_grid_search.best_params_)
print('the best score = ', np.sqrt(-lgb_grid_search.best_score_))



### Summary
the most algorithm that worked well is **lgb** then **xgb** then **RandomForestRegressor** then **DecisionTreeRegressor**

# Final Model for Test set



### Working on Test data processing


In [ ]:
test = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
test

In [ ]:
def make_data():
    global test
    n = 0
    for  i in test.segment_id :
        file = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/test/{}.csv'.format(i))
        for x in file.columns:
            test.loc[n:n+1,x+'_mean'] = file[x].mean()
            test.loc[n,x+'_std'] = file[x].std()
            test.loc[n,x+'_min'] = file[x].min()
            test.loc[n,x+'_20'] = file[x].quantile(0.20)
            test.loc[n,x+'_40'] = file[x].quantile(0.40)
            test.loc[n,x+'_50'] = file[x].quantile(0.50)
            test.loc[n,x+'_60'] = file[x].quantile(0.60)
            test.loc[n,x+'_80'] = file[x].quantile(0.80)
            test.loc[n,x+'_max'] = file[x].max()
        n +=1

    test.fillna(0,inplace=True)
    return test
make_data()    

In [ ]:
segment_id_test = test['segment_id']
test.drop(['time_to_eruption','segment_id'],axis = 1 , inplace=True)

In [ ]:
test.drop(['sensor_1_50', 'sensor_2_50', 'sensor_3_50','sensor_4_50', 'sensor_5_50', 'sensor_6_50',
                 'sensor_7_50', 'sensor_8_50', 'sensor_9_50', 'sensor_10_50'],axis = 1 , inplace=True)

In [ ]:
std_test = std_scaler.transform(test)
test = pd.DataFrame(std_test, columns = test.columns)
test

### Final Model 

### We will try *lgb* alone and ensemble *[lgb+xgb]* finally we also will try ensemble *[lgb+xgb+RandomForestRegressor]*

In [ ]:
lgb_final_model    = lgb_grid_search.best_estimator_
xgb_final_model    = xgb_grid_search.best_estimator_
forest_final_model = forest_grid_search.best_estimator_


In [ ]:
lgb_y_pred    = lgb_final_model.predict(test)
xgb_y_pred    = xgb_final_model.predict(test)
forest_y_pred = forest_final_model.predict(test)

### Submission

In [ ]:
sub = pd.DataFrame()
sub['segment_id'] = segment_id_test
sub['time_to_eruption'] = lgb_y_pred
sub.to_csv('submission_lgb.csv',index=False)

In [ ]:
sub_ensemble = pd.DataFrame()
sub_ensemble['segment_id'] = segment_id_test
sub_ensemble['time_to_eruption'] = lgb_y_pred*0.6 + xgb_y_pred*0.4
sub_ensemble.to_csv('submission_ensemble_1.csv',index=False)

In [ ]:
sub_ensemble_ = pd.DataFrame()
sub_ensemble_['segment_id'] = segment_id_test
sub_ensemble_['time_to_eruption'] = lgb_y_pred*0.60 + xgb_y_pred*0.30 + forest_y_pred*0.10
sub_ensemble_.to_csv('submission_ensemble_2.csv',index=False)